In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/lighting_images/ # 수집한 데이터셋이 있는 디렉토리 / 다음 구조와 같이 한 디렉토리에 on, off 페어가 모두 있어야 증강이 됩니다.
#  lighting_images
    #           ㄴ dataset_cloning
    #           ㄴ imgpair
                    # ㄴ1
                        # ㄴon.jpg , off.jpg
                    # ㄴ2
                        # ㄴon.jpg , off.jpg

/content/drive/MyDrive/lighting images


In [ ]:
'''
Augmentation.ipynb는 파인튜닝을 위해 저희가 수집한 on,off 또는 color,off 이미지페어를 증강하는 노트북입니다.

먼저 현재 위치한 3번셀은 cross attention튜닝을 위한 on,off데이터증강을 진행하는 곳입니다.
cross attention튜닝의 목적이 다른 조명이 있어도 저희가 학습시킨 조명에만 조명정보를 입히는것이므로
최대한 많은 데이터를 train으로 넣어 많은 epoch를 돌려 의도적으로 과적합시키는것이 적합하다고 생각하였습니다.
그래서 매우 높은 TRAIN_RATIO를 설정하였고, 과적합이 되었는지 확인할 수 있을정도만 Validation data를 구성하였습니다.
또한 더욱 다양한 구도와 입력 조명조건을 학습하기위해 증강을 해야하나 조명 고유의 분위기를 크게 해치지않는
좌우반전과 off이미지에만 밝기증강하는것이 적합하다고 생각하여 데이터증강을 진행했습니다.

그 후 4번셀에서는 cross attention튜닝이 끝난 후 Adaptor튜닝을 위한 color, off 데이터증강을 진행하는 곳입니다.
cross attention튜닝이 완료되어 이미 조명정보는 target조명에만 적용되고있는상태이므로 따로 증강을 하지않아도 무방하지만
이미지페어 자체의 수가 매우 적기때문에 학습의 안정성과 데이터부족을 해결하기 위해 좌우반전증강만 진행하였습니다.

상하반전 증강을 하지않는 이유는 저희가 선택한 조명이 뒤집혀서 input되는일이 없기때문에 상하반전은 따로 적용하지않았습니다.
'''


import os
import cv2
import numpy as np
import shutil
import random
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# ==========================================
# [설정] cross attention 학습 데이터 증강
# 좌우반전 + on / off 페어에서 off만 밝기증강
# ==========================================
SOURCE_ROOT = "./imgpair"           # 원본 데이터 폴더
OUTPUT_ROOT = "./dataset_cloning"     # 최종 저장될 폴더
TRAIN_RATIO = 0.95                   # 학습 데이터 비율 (95%)

# 증강 설정
AUGMENT_BRIGHTNESS_STEPS = 2       # 밝기 증강 단계 수

def get_all_pairs(source_dir):
    """
    imgpair 폴더 내부의 모든 폴더를 뒤져서 ON/OFF 쌍을 찾습니다.
    """
    pairs = []
    for root, dirs, files in os.walk(source_dir):
        off_f, on_f = None, None

        for f in sorted(files):
            lower = f.lower()
            if not lower.endswith(('.jpg', '.jpeg', '.png', '.avif', '.webp')): continue #사진파일만 찾기

            # 1. OFF 파일 찾기
            if 'off' in lower:
                off_f = os.path.join(root, f)
            # 2. ON 파일 찾기
            elif 'on' in lower:
                on_f = os.path.join(root, f)

        if off_f and on_f:
            pairs.append({'off': off_f, 'on': on_f})

    return pairs

def process_train_pair(args): #train은 증강
    """
    [Train용] 원본 저장 + 증강(Flip, Brightness) 수행
    """
    pair, idx, output_dir = args

    # 이미지 읽기
    img_off = cv2.imread(pair['off'])
    img_on  = cv2.imread(pair['on'])

    if img_off is None or img_on is None: return

    # 파일명 (확장자 포함)
    ext = os.path.splitext(pair['off'])[1]
    base_name = f"{idx:05d}" # 고유 ID 부여

    # -------------------------------------------------------
    # 1. 원본 저장
    # -------------------------------------------------------
    save_dir = os.path.join(output_dir, f"{base_name}_original")
    os.makedirs(save_dir, exist_ok=True)
    cv2.imwrite(os.path.join(save_dir, "off.jpg"), img_off)
    cv2.imwrite(os.path.join(save_dir, "on.jpg"), img_on)

    # -------------------------------------------------------
    # 2. 좌우 반전 (Flip)
    # -------------------------------------------------------
    flip_off = cv2.flip(img_off, 1)
    flip_on  = cv2.flip(img_on, 1)

    save_dir = os.path.join(output_dir, f"{base_name}_flip")
    os.makedirs(save_dir, exist_ok=True)
    cv2.imwrite(os.path.join(save_dir, "off.jpg"), flip_off)
    cv2.imwrite(os.path.join(save_dir, "on.jpg"), flip_on)

    # -------------------------------------------------------
    # 3. 밝기 증강 (Original & Flip 각각 적용)
    # -------------------------------------------------------
    for i in range(1, AUGMENT_BRIGHTNESS_STEPS + 1):
        percent = i * 15 # AUGMENT_BRIGHTNESS_STEPS가 2이므로 밝기 +15%, +30%
        alpha = 1.0 + (percent / 100.0)

        # (A) 원본 밝기 증강
        bright_off = cv2.convertScaleAbs(img_off, alpha=alpha, beta=0)
        save_dir = os.path.join(output_dir, f"{base_name}_orig_bright_{percent:02d}")
        os.makedirs(save_dir, exist_ok=True)
        cv2.imwrite(os.path.join(save_dir, "off.jpg"), bright_off)
        cv2.imwrite(os.path.join(save_dir, "on.jpg"), img_on) # ON은 밝기증강 시 조명 빛에 영향가기때문에 밝기증강x

        # (B) 반전 밝기 증강
        bright_flip_off = cv2.convertScaleAbs(flip_off, alpha=alpha, beta=0)
        save_dir = os.path.join(output_dir, f"{base_name}_flip_bright_{percent:02d}")
        os.makedirs(save_dir, exist_ok=True)
        cv2.imwrite(os.path.join(save_dir, "off.jpg"), bright_flip_off)
        cv2.imwrite(os.path.join(save_dir, "on.jpg"), flip_on) # ON은 밝기증강 x

def process_val_pair(args): #validation은 증강x
    """
    [Validation용] 증강 없이 원본만 복사
    """
    pair, idx, output_dir = args

    img_off = cv2.imread(pair['off'])
    img_on  = cv2.imread(pair['on'])

    if img_off is None or img_on is None: return

    base_name = f"{idx:05d}"
    save_dir = os.path.join(output_dir, f"{base_name}_original")
    os.makedirs(save_dir, exist_ok=True)

    cv2.imwrite(os.path.join(save_dir, "off.jpg"), img_off)
    cv2.imwrite(os.path.join(save_dir, "on.jpg"), img_on)

def main():
    # 1. 준비
    if os.path.exists(OUTPUT_ROOT):
        shutil.rmtree(OUTPUT_ROOT) # 기존 폴더 있으면 삭제 (클린 스타트)

    train_dir = os.path.join(OUTPUT_ROOT, "train") #OUTPUT_ROOT/train에 저장
    val_dir = os.path.join(OUTPUT_ROOT, "validation") #OUTPUT_ROOT/validation에 저장
    os.makedirs(train_dir)
    os.makedirs(val_dir)

    print(f"🚀 데이터셋 준비 시작: {SOURCE_ROOT}")

    # 2. 쌍 찾기 및 섞기
    all_pairs = get_all_pairs(SOURCE_ROOT) #SOURCE_ROOT 내부에 있는 모든 폴더를 뒤져서 on/off 페어 찾기
    random.shuffle(all_pairs) #찾은 페어를 랜덤하게 섞음

    total = len(all_pairs)
    split_idx = int(total * TRAIN_RATIO)

    train_list = all_pairs[:split_idx] # Train / Validation으로 분할
    val_list = all_pairs[split_idx:]

    print(f"📦 총 데이터: {total}쌍")
    print(f"   - Train (증강 O): {len(train_list)}쌍 -> 약 {len(train_list) * 6}개 생성 예정")
    # 원본 1쌍 > (원본 + 반전) * (밝기원본 + 2가지 밝기증강) >> 6쌍
    print(f"   - Valid (증강 X): {len(val_list)}쌍")

    # 3. 병렬 처리 실행
    num_cores = cpu_count()
    print(f"⚡️ CPU {num_cores}코어로 병렬 처리 중...")

    # (A) Train 데이터 생성
    train_tasks = [(pair, i, train_dir) for i, pair in enumerate(train_list)]
    with Pool(num_cores) as pool:
        list(tqdm(pool.imap_unordered(process_train_pair, train_tasks), total=len(train_tasks), desc="Generating Train Set"))

    # (B) Validation 데이터 생성
    val_tasks = [(pair, i, val_dir) for i, pair in enumerate(val_list)]
    with Pool(num_cores) as pool:
        list(tqdm(pool.imap_unordered(process_val_pair, val_tasks), total=len(val_tasks), desc="Generating Val Set"))

    print("\n🎉 모든 작업 완료!")
    print(f"   - 학습 데이터: {train_dir}")
    print(f"   - 검증 데이터: {val_dir}")

if __name__ == "__main__":
    main()

🚀 데이터셋 준비 시작: ./imgpair
📦 총 데이터: 300쌍
   - Train (증강 O): 285쌍 -> 약 6270개 생성 예정
   - Valid (증강 X): 15쌍
⚡️ CPU 12코어로 병렬 처리 중...


Generating Val Set: 100%|██████████| 15/15 [00:03<00:00,  4.47it/s]


🎉 모든 작업 완료!
   - 학습 데이터: ./dataset_cloning/train
   - 검증 데이터: ./dataset_cloning/validation


In [ ]:
import os
import cv2
import numpy as np
import shutil
import random
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# ==========================================
# [설정] Adaptor 학습 데이터 증강
# 각 off / color 페어를 찾은 후 train / validation으로 나눔
# train만 좌우반전 증강
# ==========================================
SOURCE_ROOT = "./lighting_images"   # 원본 데이터 폴더
OUTPUT_ROOT = "./dataset_cloning"   # 최종 저장될 폴더
TRAIN_RATIO = 0.8                   # 학습 데이터 비율 (80%)

def get_all_pairs(source_dir):
    """
    imgpair 폴더를 뒤져서 OFF, color 세트를 찾습니다.
    """
    pairs = []
    for root, dirs, files in os.walk(source_dir):
        off_f, on_f, color_f = None, None, None

        for f in sorted(files):
            lower = f.lower()
            if not lower.endswith(('.jpg', '.jpeg', '.png', '.avif', '.webp')): continue #이미지 파일만 찾기

            # 1. OFF 파일 찾기
            if 'off' in lower:
                off_f = os.path.join(root, f)

            # 2. color 파일 찾기 (_r : red, _o : orange, _y : yellow, _g : green, _b : blue, _p : purple)
            # color 파일은 123_r.jpg와 같이 색깔태그가 포함되어있음
            elif '_r' in lower and 'on' not in lower: #다른 color로 바꿀 때 여기 수정 ex) _r >> _o
                color_f = os.path.join(root, f)

        if off_f and color_f:
            pairs.append({'off': off_f,'color': color_f})

    return pairs

def process_train_pair(args):
    """
    [Train용] 원본 저장 + Flip 증강 (좌우 반전)
    """
    pair, idx, output_dir = args

    # 이미지 읽기
    img_off = cv2.imread(pair['off'])
    img_color = cv2.imread(pair['color'])

    if img_off is None or img_on is None or img_color is None: return

    base_name = f"{idx:05d}" # 고유 ID 부여

    # -------------------------------------------------------
    # 1. 원본 저장
    # -------------------------------------------------------
    save_dir = os.path.join(output_dir, f"{base_name}_original")
    os.makedirs(save_dir, exist_ok=True)
    cv2.imwrite(os.path.join(save_dir, "off.jpg"), img_off)
    cv2.imwrite(os.path.join(save_dir, "red.jpg"), img_color) #다른 color로 바꿀 때 여기 수정

    # -------------------------------------------------------
    # 2. 좌우 반전 (Flip)
    # -------------------------------------------------------
    flip_off = cv2.flip(img_off, 1)
    flip_color = cv2.flip(img_color, 1)

    save_dir = os.path.join(output_dir, f"{base_name}_flip")
    os.makedirs(save_dir, exist_ok=True)
    cv2.imwrite(os.path.join(save_dir, "off.jpg"), flip_off)
    cv2.imwrite(os.path.join(save_dir, "red.jpg"), flip_color) #다른 color로 바꿀 때 여기 수정

def process_val_pair(args):
    """
    [Validation용] 증강 없이 원본만 복사
    """
    pair, idx, output_dir = args

    img_off = cv2.imread(pair['off'])
    img_color = cv2.imread(pair['color'])

    if img_off is None or img_on is None or img_color is None: return

    base_name = f"{idx:05d}"
    save_dir = os.path.join(output_dir, f"{base_name}_original")
    os.makedirs(save_dir, exist_ok=True)

    cv2.imwrite(os.path.join(save_dir, "off.jpg"), img_off)
    cv2.imwrite(os.path.join(save_dir, "red.jpg"), img_color) #다른 color로 바꿀 때 여기 수정

def main():
    # 1. 준비
    if os.path.exists(OUTPUT_ROOT):
        shutil.rmtree(OUTPUT_ROOT) # 기존 폴더 있으면 삭제 (클린 스타트)

    train_dir = os.path.join(OUTPUT_ROOT, "train") #OUTPUT_ROOT/train에 저장
    val_dir = os.path.join(OUTPUT_ROOT, "validation")#OUTPUT_ROOT/validation에 저장
    os.makedirs(train_dir)
    os.makedirs(val_dir)

    print(f"🚀 color 데이터셋 준비 시작: {SOURCE_ROOT}")

    # 2. 쌍 찾기 및 섞기
    all_pairs = get_all_pairs(SOURCE_ROOT)

    # 데이터 확인
    if len(all_pairs) == 0:
        print("⚠️ 주의: (OFF, color) 세트를 하나도 찾지 못했습니다.")
        return

    random.shuffle(all_pairs) #찾은 off / color 페어를 섞음

    total = len(all_pairs)
    split_idx = int(total * TRAIN_RATIO)

    train_list = all_pairs[:split_idx] # train / validation을 정해진 비율로 분할
    val_list = all_pairs[split_idx:]

    print(f"📦 총 데이터: {total}쌍")
    print(f"   - Train (증강 O): {len(train_list)}쌍 -> {len(train_list) * 2}개 생성 예정")
    # 좌우반전만 하는 증강이므로 2배
    print(f"   - Valid (증강 X): {len(val_list)}쌍")

    # 3. 병렬 처리 실행
    num_cores = cpu_count()
    print(f"⚡️ CPU {num_cores}코어로 병렬 처리 중...")

    # (A) Train 데이터 생성
    train_tasks = [(pair, i, train_dir) for i, pair in enumerate(train_list)]
    with Pool(num_cores) as pool:
        list(tqdm(pool.imap_unordered(process_train_pair, train_tasks), total=len(train_tasks), desc="Generating Train Set"))

    # (B) Validation 데이터 생성
    val_tasks = [(pair, i, val_dir) for i, pair in enumerate(val_list)]
    with Pool(num_cores) as pool:
        list(tqdm(pool.imap_unordered(process_val_pair, val_tasks), total=len(val_tasks), desc="Generating Val Set"))

    print("\n🎉 모든 작업 완료!")
    print(f"   - 저장 경로: {OUTPUT_ROOT}")
    print(f"   - 파일 구성: off.jpg / color.jpg")

if __name__ == "__main__":
    main()

🚀 color 데이터셋 준비 시작: ./lighting_images
📦 총 데이터: 300쌍
   - Train (증강 O): 240쌍 -> 480개 생성 예정
   - Valid (증강 X): 60쌍
⚡️ CPU 12코어로 병렬 처리 중...


Generating Val Set: 100%|██████████| 60/60 [00:14<00:00,  4.02it/s]


🎉 모든 작업 완료!
   - 저장 경로: ./dataset_cloning
   - 파일 구성: off.jpg / on.jpg / color.jpg
